In [1]:
import pandas as pd
import os
import datetime
import re
import statsmodels.api as sm
from math import sqrt
from itertools import permutations

In [2]:
brand = 'Louis Vuitton'
directory = '/Users/yulia/Desktop/chanel/LV_Alma_Pm'
os.chdir(directory)

In [3]:
fabrics = {
    'Chanel': [['lambskin', 'lamb', 'lamskin'], ['patent', 'vernis'], ['caviar', 'cavier'], ['calfskin', 'calf'], ['tweed'],
               ['canvas'], ['velvet'], ['denim'], ['jersey'], ['suede'], ['goat skin', 'goatskin'], ['fabric'], ['python'],
               ['cotton'], ['satin'], ['nylon'], ['ostrich'], ['snakeskin']],
    'Fendi': [['calfskin', 'calf leather'], ['velvet'], ['python'], ['patent']],
    'Gucci': [['calfskin', 'calf', 'calfakin'], ['velvet'], ['canvas'], ['jacquard'], ['velure']],
    'Saint Laurent': [['calfskin', 'calf', 'calf-skin leather'], ['grained leather'], ['suede', 'suded'], ['crocodile', 'croc leather'],
                     ['smooth'], ['patent'], ['lambskin']],
    'Louis Vuitton': [['canvas'], ['epi leather', 'epi', 'epic'], ['patent leather', 'patent'], ['canvas leather'],
                      ['vernis leather', 'vernis']]
}

sizes = {
    'Chanel': {
        'Medium': (10, 3, 6.3, 3)
    },
    'Louis Vuitton': {
        'Pm': (12.6, 9.4, 6, 5)
    },
    'Saint Laurent': {
        'Small': (12.5, 9.8, 6.4, 5)
    },
    'Gucci' : {
        'Handle': (12.5, 8.5, 4.25, 4),
        'Mini': (7.5, 5.5, 3, 4),
        'Shoulder': (10, 6.75, 3, 4)
    },
    'Fendi': {
        'Small': (5.2, 7.5, 3.5, 3),
        'Medium': (7.3, 9.8, 4.3, 3)
    }
}

In [4]:
df = pd.DataFrame()
for file in [file for file in os.listdir() if file.endswith('.csv')]:
    MONTH, DAY = int(re.search('\_(\d+)\_(\d+)', file)[1]), int(re.search('\_(\d+)\_(\d+)', file)[2])
    df1 = pd.read_csv(file)
    df1['sc_date'] = datetime.date(year=2019, month=MONTH, day=DAY)
    df = pd.DataFrame(df.append(df1))

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


# clean dataset

In [5]:
df = df[df['bags_brand'] == brand]
size_before = df.shape[0]

In [6]:
def material(string, brand):
    list_of_materials = []
    string_refined = str(string).lower()
    for material_list in fabrics[brand]:
        for material in material_list:
            r = r'\b%s\b'%material
            if re.search(r, string_refined):
                list_of_materials.append(material_list[0])
    if not list_of_materials:
        if 'leather' in string_refined:
            list_of_materials.append('leather')
    if list_of_materials:
        return str(list_of_materials)
    return

In [7]:
def material_refine(row, brand):
    for string in [row['bags_name'], row['bags_description'], row['bags_fabric']]:
        found_materials = material(string, brand)
        if found_materials:
            return found_materials
    return 'Other'

In [8]:
if brand in fabrics:
    df['materials_list'] = df.apply(lambda x: material_refine(x, brand), axis = 1)

In [9]:
def extract_price(text_string):
    if type(text_string) == str:
        try:
            digits = re.search('\$(.*)\.\d{2}$', text_string)[1]
            digits = ''.join(digits.split(','))
            return int(digits)
        except:
            return 'Error'
    return

In [10]:
df['bags_price_refined'] = df['bags_price'].map(lambda x: extract_price(x))
df['sold_price_refined'] = df['bags_sold_for'].map(lambda x: extract_price(x))
df['retail_price_refined'] = df['bags_retail_price'].map(lambda x: extract_price(x))

In [11]:
def bag_id(bag_dict):
    if type(bag_dict['bags_on_sale']) == str:
        res = re.search('\/(\d{5,})\/', bag_dict['bags_on_sale'])
        if res:
            return res[1]
    elif type(bag_dict['bags_sold']) == str:
        res = re.search('\/(\d{5,})\/', bag_dict['bags_sold'])
        if res:
            return res[1]
    else:
        return

In [12]:
df['id'] = df.apply(lambda x: bag_id(x), axis = 1)

In [13]:
def popular(text):
    number = str(text).split(' ')[0]
    if number.isdigit():
        return number
    else:
        return 0

In [14]:
df['likes'] = df['bags_popular'].map(popular)

In [15]:
def extract_size(text):
    try:
        l = float(re.search('(\d*\.*\d*)"L', text)[1])
        w = float(re.search('(\d*\.*\d*)"W', text)[1])
        h = float(re.search('(\d*\.*\d*)"H', text)[1])
        return l, w, h
    except:
        return (100, 100, 100)

In [16]:
def vector_distance(x, y):
    if len(x) == len(y):
        return sqrt(sum((x[i] - y[i])**2 for i in range(len(x))))
    else:
        raise Exception('Different dimension of vectors')

In [17]:
def correct_size(text):
    min_diff = 1000
    correct_size = None
    for size in sizes[brand]:
        l, w, h, diff = sizes[brand][size]
        L, W, H = extract_size(text)
        cur_diff = min([vector_distance((L,W,H), c) for c in permutations([l, w, h])])
        if cur_diff < diff and cur_diff < min_diff:
            min_diff = cur_diff
            correct_size = size
    return correct_size

In [18]:
#filter out incorrect sizes
df['size'] = df.apply(lambda x: 
                 correct_size(x['bags_size']) or correct_size(x['bags_fabric']) or 'Other', axis = 1)

In [19]:
df['size'].value_counts()

Pm       140246
Other     24142
Name: size, dtype: int64

In [20]:
df['id'] = df['id'].astype(int)
df['likes'] = df['likes'].astype(int)
df['bags_color'] = df['bags_color'].fillna('Other')

In [21]:
print("{0:.0%}".format(df.shape[0] / size_before))

100%


In [22]:
export_columns = ['bags_brand', 'id', 'sc_date', 'bags_color', 'bags_condition', 'materials_list', 'bags_price_refined',
                 'sold_price_refined', 'retail_price_refined', 'likes', 'size']

In [23]:
if brand == 'Gucci':
    df = df[df['sc_date'].map(lambda x: datetime.date(2019,2,13)<=x<=datetime.date(2019,5,10))]


In [24]:
df.reset_index()[export_columns].to_pickle('%s.pkl' % brand)

In [25]:
# df.reset_index()[export_columns].to_csv('refined_data_example.csv')

In [26]:
pd.pivot_table(df, values='id', index='sc_date', aggfunc='count').reset_index().sort_values('sc_date')

,sc_date,id
0,2019-02-11,1797
1,2019-02-12,1774
2,2019-02-13,1830
3,2019-02-14,1805
4,2019-02-15,1801
...,...,...
84,2019-05-07,1929
85,2019-05-08,1910
86,2019-05-09,1928
87,2019-05-10,1920
